In [1]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.8 MB/s eta 0:00:00


In [2]:
# Importing packages and data
from time import time
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.svm import LinearSVC
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix


# from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import (RandomForestClassifier,
                              AdaBoostClassifier,
                              GradientBoostingClassifier,
                              HistGradientBoostingClassifier)
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [4]:
df_train = pd.read_csv('train_set.csv')
df_test = pd.read_csv('test_set.csv')


In [5]:
%%time
# Preprocessing our data
# Convert a collection of raw documents to a matrix of TF-IDF features.
tfidf = TfidfVectorizer(# analyzer='word', # default='word'
                        ngram_range=(1, 3), #(2, 6), # 3,6
                        # max_df=1,
                        min_df=2, # 10
                        max_features=2000, #300, 500, 10000 (thsi will limit the vocavulary)
                        # sublinear_tf=True,
                        use_idf=True
                        )

CPU times: user 63 µs, sys: 7 µs, total: 70 µs
Wall time: 74.4 µs


In [6]:
%%time
# numericalization of the text into numbers
X_train = tfidf.fit_transform(df_train['transcript']).toarray()
X_test = tfidf.transform(df_test['transcript']).toarray()

y_train, y_test = df_train['label'], df_test['label']

# slip the train set and reseve 20% of the data for the evaluation
X_train_sub, X_valid, y_train_sub, y_valid = train_test_split(X_train,
                                                              y_train,
                                                              test_size=0.2,
                                                              random_state=42,
                                                              shuffle= True
                                                              )


CPU times: user 32.1 s, sys: 1.3 s, total: 33.4 s
Wall time: 38 s


In [7]:
tfidf

TfidfVectorizer(max_features=2000, min_df=2, ngram_range=(1, 3))

In [8]:
# check the shape of each set
print("X_train shape: {}".format(X_train_sub.shape))
print("X_val shape: {}".format(X_valid.shape))
print("X_test shape: {}".format(X_test.shape))

print("y_train shape: {}".format(y_train_sub.shape))
print("y val shape: {}".format(y_valid.shape))
print("y_test shape: {}".format(y_test.shape))

X_train shape: (1872, 2000)
X_val shape: (469, 2000)
X_test shape: (440, 2000)
y_train shape: (1872,)
y val shape: (469,)
y_test shape: (440,)


In [9]:
print(X_train_sub)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.0415772  0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.27647962 0.03071996]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [10]:
X_test

array([[0.        , 0.        , 0.05608251, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [11]:
# Setting up our results dataframe
df_results = pd.DataFrame(columns=['F1_score', 'Precision', 'Recall', 'Accuracy', 'Accuracy1', 'Training time', 'Test time'])

In [12]:

#llogistic regression
lr = LogisticRegression()
lr

LogisticRegression()

In [13]:
#Decision Tree
dt = DecisionTreeClassifier()

DecisionTreeClassifier()

In [14]:
#Random Forest
rf = RandomForestClassifier()

RandomForestClassifier()

In [15]:
#Adaboost
# base_estim = DecisionTreeClassifier(max_depth=1, max_features=0.06)
ab = AdaBoostClassifier()

AdaBoostClassifier()

In [16]:
#Gradient Boosted Decision Trees (Scikit-Learn)
gbm = GradientBoostingClassifier()

GradientBoostingClassifier()

In [17]:
#XGBoost
xgb = XGBClassifier()

/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The XGBClassifier or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


AttributeError: 'super' object has no attribute '__sklearn_tags__'

AttributeError: 'super' object has no attribute '__sklearn_tags__'

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [18]:
#LightGBM
lgbm = LGBMClassifier()

/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The LGBMClassifier or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


LGBMClassifier()

In [19]:
#CatBoost
cb = CatBoostClassifier()

In [20]:
#Bonus: Histogram-based Gradient Boosting Machine (Scikit-Learn)
hgbm = HistGradientBoostingClassifier()

HistGradientBoostingClassifier()

In [42]:
## Testing LinearSVC
svc = LinearSVC()

In [44]:

#Comparing our Competitors
models = [lr, dt, rf, ab, hgbm, lgbm, xgb, svc]
# models = [rf, xgb, lgbm, cb]
model_names = [i.__class__.__name__ for i in models]
model_names

['LogisticRegression',
 'DecisionTreeClassifier',
 'RandomForestClassifier',
 'AdaBoostClassifier',
 'HistGradientBoostingClassifier',
 'LGBMClassifier',
 'XGBClassifier',
 'LinearSVC']

In [45]:
# Setting up our results dataframe
df_results = pd.DataFrame(columns=['Accuracy', 'Precision', 'Recall', 'F1-Score', 'Training time', 'Test time'])

In [48]:
from timeit import default_timer as timer

start = timer()
#Train each of our candidate models
for m, n in zip(models, model_names):
  train_time = 0
  test_time = 0

  if n == 'CatBoostClassifier' or n=='GradientBoostingClassifier':
      start_train = time()
      m.fit(X_train_sub,
            y_train_sub,
            eval_set = [(X_valid, y_valid)],
            early_stopping_rounds=15,
            verbose=1)
      train_time = time() - start_train

  else:
      start_train = time()
      m.fit(X_train, y_train)
      train_time = time() - start_train

  # Make test of the model
  start_test = time()
  predicted = m.predict(X_test)
  test_time = time() - start_test


  report = classification_report(y_test, predicted, output_dict=True)



  df_results.loc[n] = [report['accuracy'], report['macro avg']['precision'], report['macro avg']['recall'], report['macro avg']['f1-score'], train_time, test_time]

  del m

train_test_time = timer() - start
print('The time is {:.4f} seconds'.format(train_test_time))

/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 557, number of negative: 1784
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.064178 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 180315
[LightGBM] [Info] Number of data points in the train set: 2341, number of used features: 2000
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.237933 -> initscore=-1.164048
[LightGBM] [Info] Start training from score -1.164048
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


The time is 51.4170 seconds


In [49]:
df_results

,Accuracy,Precision,Recall,F1-Score,Training time,Test time
LogisticRegression,1.000000,1.000000,1.000000,1.000000,0.114818,0.002258
DecisionTreeClassifier,0.995455,0.993746,0.993746,0.993746,0.739660,0.002096
RandomForestClassifier,1.000000,1.000000,1.000000,1.000000,2.175025,0.024515
AdaBoostClassifier,1.000000,1.000000,1.000000,1.000000,9.577715,0.072500
HistGradientBoostingClassifier,1.000000,1.000000,1.000000,1.000000,23.193649,0.024073
LGBMClassifier,1.000000,1.000000,1.000000,1.000000,7.212148,0.010477
XGBClassifier,1.000000,1.000000,1.000000,1.000000,8.101985,0.006690
LinearSVC,1.000000,1.000000,1.000000,1.000000,0.057216,0.002043
